In [23]:
# ignore warnings.
import warnings
warnings.filterwarnings('ignore')

# Audio processing
from scipy.io import wavfile

# maths and sci libraries.
import numpy as np
import scipy as sp

# for dict and all.
import collections

# for plotting.
import matplotlib.pyplot as plt

# signal processing
from scipy.io                     import wavfile
from scipy                        import stats, signal
from scipy.fftpack                import fft

from scipy.signal                 import lfilter, hamming
from scipy.fftpack.realtransforms import dct
from scikits.talkbox              import segment_axis
from scikits.talkbox.features     import mfcc


# encoding purpose.
from base64 import b64decode

#pandas for csvs.
import pandas as pd

# import stft
import stft

In [24]:
# create the audio files path.
audio_file = collections.defaultdict(dict)

# initialise the audio songs as (genre, path).
# where path is path of the current .wav audio.

audio_file["rock"]["path"] = r"tomydeepestego.wav"

In [25]:
def audio_length():
    samplerate, wavedata = wavfile.read(audio_file["rock"]["path"])
    audio_file["rock"]["wavedata"] = wavedata
    audio_file["rock"]["samplerate"] = samplerate
    number_of_samples = wavedata.shape[0]
    print samplerate, wavedata.shape[0]
    # song length : number of samples/samplerate.
    print "Audio length: " + str(number_of_samples/samplerate) + " seconds"


In [26]:
audio_length()

44100 16181864
Audio length: 366 seconds


In [27]:
def wavedata_mean():
    audio_file["rock"]["wavedata"] = np.mean(audio_file["rock"]["wavedata"], axis=1)

In [28]:
wavedata_mean()

In [29]:
""" Zero Crossing Rate : Is a time domain feature.
    Simple, straightforward and inexpensive feature to examine the similarity between two sets of time series.
    It is the number of times signal changes sign. It is useful for signals affected by noise.
"""
def zero_crossing_rate_bruteForce(wavedata):
    zero_crossing = 0
    for i in range(1, number_of_samples):
        if (wavedata[i-1] < 0 and wavedata[i]>0) or (wavedata[i-1] > 0 and wavedata[i] < 0) or (wavedata[i-1] != 0 and wavedata[i] == 0):
            zero_crossing += 1;
    zero_crossing_rate = zero_crossing / float(number_of_samples-1)
    return zero_crossing_rate


def zero_crossing_rate(wavedata, block_length, sample_rate):
    # Number of blocks required.
    num_blocks = int(np.ceil(len(wavedata)/block_length))
    
    # Timestamps for the beginning of the blocks.
    timestamps = (np.arange(0, num_blocks - 1) * (block_length/float(sample_rate)))
    
    zcr = []
    for i in range(0, num_blocks - 1):
        start = i*block_length
        stop = np.min([(start + block_length - 1), len(wavedata)])
        zc = 0.5*np.mean(np.abs(np.diff(np.sign(wavedata[start:stop]))))
        zcr.append(zc)
        
    return np.asarray(zcr), np.asarray(timestamps)


In [30]:
zcr, zcr_timestamps = zero_crossing_rate(audio_file["rock"]["wavedata"], 1024, audio_file["rock"]["samplerate"])

In [31]:
zcr

array([ 0.        ,  0.        ,  0.        , ...,  0.02446184,
        0.04354207,  0.03620352])

In [33]:
plt.plot(zcr_timestamps, zcr)

In [34]:
""" Root mean Square : comparing arbitary waveforms based upon 
    their equivalent energy."""
def root_mean_square(wavedata, block_length, sample_rate):
    num_blocks = int(np.ceil(len(wavedata)/block_length))
    
    timestamps = (np.arange(0, num_blocks-1) * (block_length/float(sample_rate)))
    
    rms = []
    
    for i in range(0, num_blocks-1):
        start = i*block_length
        stop = np.min([(start + block_length -1), len(wavedata)])
        
        rms_seg = np.sqrt(np.mean(wavedata[start:stop]**2))
        rms.append(rms_seg)
    return np.asarray(rms), np.asarray(timestamps)

In [35]:
rms, rms_timestamps = root_mean_square(audio_file["rock"]["wavedata"], 1024, audio_file["rock"]["samplerate"])

In [36]:
plt.plot(rms_timestamps, rms)

In [37]:
""" Spectral features. """
""" Spectral centroid : centre of gravity.
    Tells the frequency around which most of the signal energy is concentrated.
    Tells how dark/bright the sound is."""
def spectral_centroid(wavedata, window_size, sample_rate):
    magnitude_spectrum = stft.spectrogram(wavedata, window_size)
    timebins, freqbins = np.shape(magnitude_spectrum)
    timestamps = (np.arange(0, timebins - 1)*(timebins/float(sample_rate)))
    spec_centroid = []
    
    for t in range(timebins - 1):
        power_spectrum = np.abs(magnitude_spectrum[t])**2
        
        sc_t = np.sum(power_spectrum * np.arange(1, freqbins + 1))/np.sum(power_spectrum)
        
        spec_centroid.append(sc_t)
        
    return np.nan_to_num(np.asarray(spec_centroid)), np.asarray(timestamps)


In [38]:
spec_centroid, spec_ts = spectral_centroid(audio_file["rock"]["wavedata"], 1024, audio_file["rock"]["samplerate"])

In [39]:
plt.plot(spec_ts, spec_centroid)

In [40]:
""" Sprectral rolloff :
    Nth percentile of the power spectral distribution, 
    where N is 85% or 95%. The rolloff point is the frequency below which 
    the N% of the magnitude distribution is concentrated.
    Used to distinguish voice speech from unvoiced.
    Unvoiced has a high proportion of energy contained in the high-frequency range of the spectrum.
    - fraction of bins in the power spectrum at which 85%(N%) of the power is at lower frequencies.
"""
def spectral_rolloff(wavedata, window_size, sample_rate, k=0.85):
    # convert into frequency domain using short term fourier transform.
    magnitude_spectrum = stft.spectrogram(wavedata, window_size)
    time_bins, freq_bins = np.shape(magnitude_spectrum)
    power_spectrum = np.abs(magnitude_spectrum)**2
    
    # create timestamps
    timestamps = (np.arange(0, time_bins - 1)*(time_bins/float(sample_rate)))
        
    spec_rolloff = []
    
    spectral_sum = np.sum(power_spectrum, axis=1)
    
    for t in range(time_bins - 1):
        # find frequency-bin indices where cummulative sum of all bins is higher than k-percent of the sum of all bins.
        # minimum index = rolloff.
        spec_rolloff_temp = np.where(np.cumsum(power_spectrum[t, :]) >= k*spectral_sum[t])[0][0]
        spec_rolloff.append(spec_rolloff_temp)
    
    spec_rolloff = np.asarray(spec_rolloff).astype(float)
    
    spec_rolloff = (spec_rolloff/freq_bins)*(sample_rate/2.0)
    
    return spec_rolloff, np.asarray(timestamps)
        


In [41]:
spec_rolloff, spec_ts = spectral_rolloff(audio_file["rock"]["wavedata"], 1024, audio_file["rock"]["samplerate"])

In [42]:
plt.plot(spec_ts, spec_rolloff)


In [43]:
""" Spectral flux : squared diff in frequency distribution of two successive time frames."""
""" Helps in measuring rate of local change in the spectrum"""
def spectral_flux(wavedata, window_size, sample_rate):
    magnitude_spectrum = stft.spectrogram(wavedata, window_size)
    time_bins, freq_bins = np.shape(magnitude_spectrum)
    
    # create timestamps.
    timestamps = (np.arange(0, time_bins - 1) * (time_bins/float(sample_rate)))
    
    spec_flux = np.sqrt(sp.sum(np.diff(np.abs(magnitude_spectrum))**2, axis = 1))/freq_bins
    
    return spec_flux[1:], np.asarray(timestamps)



In [44]:
spec_flux, spec_flux_ts = spectral_flux(audio_file["rock"]["wavedata"], 1024, audio_file["rock"]["samplerate"])

In [45]:
plt.plot(spec_flux_ts, spec_flux)

In [50]:
""" MFCC : coefficients that collectively make up an MFC.
    They are derived from a type of cepstral representation of the audio ( a nonlinear spectrum of a spectrum).
    In MFC the frequency bands are equally spaced on the mel scale, which approximates the human auditory systems response more closely than the linear spaced frequency bands.
"""
def MFCC_Cal(input_data):
    # apply pre-filtering.
    
    # params
    nwin = 256
    nfft = 1024
    fs = 16000
    nceps = 13
    
    # pre-emphasis factor
    prefac = 0.97
    
    over = nwin - 160
    
    filtered_data = lfilter([1., -prefac], 1, input_data)
    
    # compute the spectrum amplitude by windowing with a hamming window.
    windows = hamming(256, sym = 0)
    framed_data = segment_axis(filtered_data, nwin, over) * windows
    
    magnitude_spectrum = np.abs(fft(framed_data, nfft, axis = -1))
    
    
    # filter the signal in the spectral domain with a triangular filter-bank, 
    # whose filters are approximately linearly spaced on the mel scale, and have equal bandwidth in the mel scale/

    lowfreq = 133.33
    linsc = 200/3
    logsc = 1.0711703
    fs = 44100
    
    nlinfilt = 13
    nlogfilt = 27
    
    #total filters 
    nfilt = nlinfilt + nlogfilt
    
    # Compute the filter bank.
    # compute start/middle/end points of the triangular filters in spectral.
    
    #domain.
    freqs = np.zeros(nfilt + 2)
    freqs[:nlinfilt] = lowfreq + np.arange(nlinfilt) * linsc
    
    freqs[nlinfilt:] = freqs[nlinfilt - 1] * logsc ** np.arange(1, nlogfilt + 3)
    
    heights = 2./(freqs[2:] - freqs[0:-2])
    
    #compute filterbank coeff (in fft domain, in bins)
    filterbank = np.zeros((nfilt, nfft))
    
    # FFT bins (in Hz)
    nfreqs = np.arange(nfft)/(1. * nfft)*fs
    
    for i in range(nfilt):
        low = freqs[i]
        cen = freqs[i+1]
        hi = freqs[i+2]
        
        lid = np.arange(np.floor(low*nfft/fs) + 1,
                        np.floor(cen*nfft/fs) + 1, dtype = np.int)
        
        rid = np.arange(np.floor(cen*nfft/fs) + 1,
                        np.floor(hi*nfft/fs) + 1, dtype = np.int)
        
        lslope = heights[i]/(cen - low)
        rslope = heights[i]/(hi-cen)
        
        filterbank[i][lid] = lslope * (nfreqs[lid] - low)
        filterbank[i][rid] = rslope * (hi - nfreqs[rid])
        
        # filter the spectrum through the triangle filterbank.
        
        mspec = np.log10(np.dot(magnitude_spectrum, filterbank.T))
        
        
        # Use the DCT to 'compress' the coefficients (spectrum -> cepstrum domain)
        MFCCs = dct(mspec, type = 2, norm = 'ortho', axis = -1)[:, :nceps]
        
        
        return MFCCs, mspec, magnitude_spectrum
    


In [51]:
MFCCs, mspec, spec = MFCC_Cal(audio_file["rock"]["wavedata"])


In [ ]:
""" Rhythm patterns : """